# AR6 calibration of FaIR 2.1

Constraint
- AR6 assessed GSAT, 1850-2020
- AR6 assessed OHC, 1971-2018
- CO2 concentration, 1750-2014

## Basic imports

In [ ]:
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
import scipy.optimize
from tqdm import tqdm

from fair21.energy_balance_model import EnergyBalanceModel

In [ ]:
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
samples = 200000
#batch_size = 1000

## Load in results

In [ ]:
temp_in = np.load('../data/ar6_ensemble_batches/temperature_1850-2030_20220415.npy')
ohc_in = np.load('../data/ar6_ensemble_batches/ohc_1971-2018_20220415.npy')
faer_in = np.load('../data/ar6_ensemble_batches/faer_2000-2019_20220415.npy')
fari_in = np.load('../data/ar6_ensemble_batches/fari_1750-2019_20220415.npy')
faci_in = np.load('../data/ar6_ensemble_batches/faci_1750-2019_20220415.npy')
co2_in = np.load('../data/ar6_ensemble_batches/co2_2014_20220415.npy')

In [ ]:
pl.hist(faer_in[5:15,:].mean(axis=0), bins=np.arange(-3.2,0.7,0.1));
np.percentile(faer_in[5:15,:].mean(axis=0), (5, 50, 95))

In [ ]:
pl.hist(fari_in[255:265,:].mean(axis=0), bins=np.arange(-1.0,0.4,0.05));
np.percentile(fari_in[255:265,:].mean(axis=0), (5, 50, 95))

In [ ]:
pl.hist(faci_in[255:265,:].mean(axis=0), bins=np.arange(-2.5,0.4,0.05));
np.percentile(faci_in[255:265,:].mean(axis=0), (5, 50, 95))

In [ ]:
faci_in[0,:]

In [ ]:
temp_in.shape

## 1. sufrace temperature 1850-2020

In [ ]:
df_gmst = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

In [ ]:
df_four = pd.read_csv('../data/forcing/AR6_GMST_four_datasets.csv', index_col=0)
df_four

In [ ]:
def rmse(obs, mod):
    return np.sqrt(np.sum((obs-mod)**2)/len(obs))

In [ ]:
rmse_temp = np.zeros((samples))

In [ ]:
temp_in-temp_in[:51,:].mean(axis=0)
#df_temp.shape

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.min(temp_in-temp_in[:51,:].mean(axis=0), axis=1), 
    np.max(temp_in-temp_in[:51,:].mean(axis=0), axis=1),
    color='#000000',
    alpha=0.03,
)
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.percentile(temp_in-temp_in[:51,:].mean(axis=0), 5, axis=1), 
    np.percentile(temp_in-temp_in[51,:].mean(axis=0), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.percentile(temp_in-temp_in[:51,:].mean(axis=0), 16, axis=1), 
    np.percentile(temp_in-temp_in[:51,:].mean(axis=0), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1850.5, 2031), 
    np.median(temp_in-temp_in[:51,:].mean(axis=0), axis=1), 
    color='#000000',
)
ax.plot(np.arange(1850.5, 2021), gmst, color='b')
ax.set_xlim(1850,2030)
ax.set_ylim(-1, 3)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Temperature anomaly - unconstrained')

In [ ]:
# pandas is so slow!
hadcrut = df_four['HadCRUT'].values
noaa = df_four['NOAA'].values
berkeley = df_four['Berkeley'].values
kadow = df_four['Kadow'].values

In [ ]:
rmse_temp = np.zeros((samples))
rmse_hadcrut = np.zeros((samples))
rmse_noaa = np.zeros((samples))
rmse_berkeley = np.zeros((samples))
rmse_kadow = np.zeros((samples))

for i in tqdm(range(samples)):
    rmse_temp[i] = rmse(gmst[:171], temp_in[:171,i]-temp_in[:51, i].mean())
    rmse_hadcrut[i] = rmse(hadcrut, temp_in[:171,i]-temp_in[:51, i].mean())
    rmse_noaa[i] = rmse(noaa, temp_in[:171,i]-temp_in[:51, i].mean())
    rmse_berkeley[i] = rmse(berkeley, temp_in[:171,i]-temp_in[:51, i].mean())
    rmse_kadow[i] = rmse(kadow, temp_in[:171,i]-temp_in[:51, i].mean())

In [ ]:
#accept_temp=(rmse_temp<0.135)
accept_temp=(rmse_temp<0.15)
#accept_temp = np.logical_or.reduce((rmse_hadcrut<0.142, rmse_noaa<0.142, rmse_berkeley<0.142, rmse_kadow<0.142))
print(np.sum(accept_temp))
valid_temp = np.arange(samples, dtype=int)[accept_temp]

In [ ]:
# pl.plot(np.arange(0.6,1.11,0.01), scipy.stats.skewnorm.pdf(np.arange(0.6,1.11,0.01), 0, scale=0.1, loc=0.85))

In [ ]:
# scipy.stats.skewnorm.ppf((0.05, 0.50, 0.95), 0, loc=.85, scale=0.1)

In [ ]:
# def opt(x, q05_desired, q50_desired, q95_desired):
#     "x is (a, loc, scale) in that order."
#     q05, q50, q95 = scipy.stats.skewnorm.ppf((0.05, 0.50, 0.95), x[0], loc=x[1], scale=x[2])
#     #print(q05, q50, q95, x)
#     return (q05-q05_desired, q50-q50_desired, q95-q95_desired)
    
# scipy.optimize.root(opt, (-0.1,0.83,0.08), (0.69, 0.85, 0.98), options={'maxfev': 5000})

In [ ]:
# scipy.stats.skewnorm.ppf((0.05, 0.50, 0.95), -1.65506091, loc=0.92708099, scale=0.12096636)

In [ ]:
# # d(x)
# pl.plot(np.linspace(0.55,1.15), scipy.stats.skewnorm.pdf(np.linspace(0.55, 1.15), -1.65506091, loc=0.92708099, scale=0.12096636))

In [ ]:
# # f(x)
# pl.hist(temp_in[245:265,:].mean(axis=0)-temp_in[100:151,:].mean(axis=0), bins=np.arange(0.2,1.41,0.02))
# #pl.xlim(0.5,1.1)

In [ ]:
# f_x = 0.01 * np.histogram(temp_in[245:265,:].mean(axis=0)-temp_in[100:151,:].mean(axis=0), bins=np.arange(0.2,1.41,0.01), density=True)[0]

In [ ]:
# d_x = np.zeros(120)
# dtemp=0.01
# for i, temp_int in enumerate(np.arange(0.2, 1.4, 0.01)):
#     d_x[i] = (
#         scipy.stats.skewnorm.cdf(temp_int+dtemp, -1.65506091, loc=0.92708099, scale=0.12096636) - 
#         scipy.stats.skewnorm.cdf(temp_int, -1.65506091, loc=0.92708099, scale=0.12096636)
#     )

In [ ]:
# accept_ratio = d_x/f_x / (np.max(d_x/f_x))
# pl.plot(np.arange(0.2, 1.4, 0.01), accept_ratio)

In [ ]:
# accept_prob = scipy.stats.uniform.rvs(loc=0, scale=1, size=samples, random_state=7324789)
# accept_temp = np.zeros(samples, dtype=bool)

# accept_ratio>=accept_prob = 

In [ ]:
# accept_prob = scipy.stats.uniform.rvs(loc=0, scale=1, size=samples, random_state=7324789)
# accept_temp = np.zeros(samples, dtype=bool)

# likelihood = scipy.stats.skewnorm.pdf(
#     temp_in[245:265,:].mean(axis=0)-temp_in[100:151, :].mean(axis=0),
#     -1.65506091, loc=0.92708099, scale=0.12096636
# )/4.5 #st.norm.pdf(-1.53, loc=-1.53, scale=0.10)

# accept_temp[likelihood>=accept_prob] = True

# np.sum(accept_temp)

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.min(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), axis=1), 
    np.max(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.percentile(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), 5, axis=1), 
    np.percentile(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.percentile(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), 16, axis=1), 
    np.percentile(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1850.5, 2031), 
    np.median(temp_in[:,accept_temp]-temp_in[:51,accept_temp].mean(axis=0), axis=1), 
    color='#000000',
)

#ax.plot(np.arange(1850.5, 2021), gmst, color='b')
ax.plot(np.arange(1850.5, 2021), df_four['HadCRUT'], color='r')
ax.plot(np.arange(1850.5, 2021), df_four['NOAA'], color='y')
ax.plot(np.arange(1850.5, 2021), df_four['Berkeley'], color='g')
ax.plot(np.arange(1850.5, 2021), df_four['Kadow'], color='c')

ax.set_xlim(1850,2030)
ax.set_ylim(-0.6, 2.2)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Temperature anomaly - constrained')

In [ ]:
# ch7 SM: aiming on 0.67 0.85 0.98
pl.hist(temp_in[145:165,accept_temp].mean(axis=0)-temp_in[:51,accept_temp].mean(axis=0))
np.percentile(temp_in[145:165,accept_temp].mean(axis=0)-temp_in[:51,accept_temp].mean(axis=0), (5, 50, 95))

In [ ]:
np.median(temp_in[180,accept_temp]-temp_in[:51,accept_temp].mean(axis=0))

## 2. Ocean heat content 1971-2018

In [ ]:
ohc_df = pd.read_csv("../data/forcing/AR6_OHC_ensemble_FGDprelim.csv", skiprows=1)
ohc = ohc_df['Central Estimate Full-depth'].values
ohc_onesigma = ohc_df['Full-depth Uncertainty (1-sigma)'].values

In [ ]:
ohc

In [ ]:
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)

In [ ]:
pl.fill_between(np.arange(1971.5, 2019), (ohc - NINETY_TO_ONESIGMA*ohc_onesigma)/0.9, (ohc + NINETY_TO_ONESIGMA*ohc_onesigma)/0.9)
pl.plot(np.arange(1971.5, 2019), ohc/0.9, color='k')

In [ ]:
((ohc[-1]-ohc[0]))/.91

In [ ]:
((ohc[-1]-ohc[0]) - NINETY_TO_ONESIGMA*np.sqrt(ohc_onesigma[0]**2+ohc_onesigma[-1]**2))/.91

In [ ]:
((ohc[-1]-ohc[0]) + NINETY_TO_ONESIGMA*np.sqrt(ohc_onesigma[0]**2+ohc_onesigma[-1]**2))/.91

In [ ]:
((ohc[-1]-ohc[0]))

In [ ]:
(ohc_in[-1,:] - ohc_in[0,:]).shape

In [ ]:
pl.hist((ohc_in[-1,:] - ohc_in[0,:]), bins=np.linspace(0e23, 6e23));

In [ ]:
ohc_to_eshc_factor = 0.91

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.min(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), axis=1), 
    np.max(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), axis=1),
    color='#000000',
    alpha=0.03,
)
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.percentile(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), 5, axis=1), 
    np.percentile(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.percentile(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), 16, axis=1), 
    np.percentile(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1971.5, 2019), 
    np.median(ohc_to_eshc_factor*(ohc_in-ohc_in[0, :]), axis=1), 
    color='#000000',
)
ax.plot(np.arange(1971.5, 2019), ohc*1e21, color='b')
ax.set_xlim(1971, 2019)
ax.set_ylim(-1e24, 1.5e24)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Ocean heat content - unconstrained')

In [ ]:
rmse(ohc, ohc_in[:,i]-ohc_in[0, i])

In [ ]:
# asymmetical bounds here are deliberate
accept_ohc = np.logical_and(
    (
        ((ohc[-1]-ohc[0]) - 0.90*NINETY_TO_ONESIGMA*np.sqrt(ohc_onesigma[0]**2+ohc_onesigma[-1]**2)) < 
        ohc_to_eshc_factor*(ohc_in[-1,:] - ohc_in[0,:])*1e-21
    ), (
        ohc_to_eshc_factor*(ohc_in[-1,:] - ohc_in[0,:])*1e-21 < 
        ((ohc[-1]-ohc[0]) + NINETY_TO_ONESIGMA*np.sqrt(ohc_onesigma[0]**2+ohc_onesigma[-1]**2))
    )
)
valid_ohc = np.arange(samples, dtype=int)[accept_ohc]
print(np.sum(accept_ohc))

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.min(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), axis=1), 
    np.max(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), axis=1),
    color='#000000',
    alpha=0.03,
)
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), 5, axis=1), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), 16, axis=1), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1971.5, 2019), 
    np.median(ohc_to_eshc_factor*(ohc_in[:, accept_ohc]-ohc_in[0, accept_ohc]), axis=1), 
    color='#000000',
)
ax.plot(np.arange(1971.5, 2019), ohc*1e21, color='b')
ax.set_xlim(1971,2019)
#ax.set_ylim(-1, 3)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Ocean heat content - constrained')

In [ ]:
# target is 329 396 463 for 16, 50, 84
pl.hist(ohc_to_eshc_factor*(ohc_in[-1, accept_ohc]-ohc_in[0, accept_ohc]));
np.percentile(ohc_to_eshc_factor*(ohc_in[-1, accept_ohc]-ohc_in[0, accept_ohc]), (5, 16, 50, 84, 95))

### 3. CO2 concentrations in 2014

In [ ]:
co2_2014 = df_conc[
    (df_conc['Variable']=='Atmospheric Concentrations|CO2') &
    (df_conc['Scenario']=='historical') &
    (df_conc['Region']=='World')
]['2014'].values[0]

In [ ]:
accept_co2 = np.logical_and((co2_2014 - 3*0.36) < co2_in, co2_in < (co2_2014 + 3*0.36))
print(np.sum(accept_co2))
valid_co2 = np.arange(samples, dtype=int)[accept_co2]

## 4. just for now, constraint on aerosol forcing

In [ ]:
#accept_faer = faer_in[5:15, :].mean(axis=0)>-2.1
#print(np.sum(accept_faer))
#valid_faer = np.arange(samples, dtype=int)[accept_faer]

In [ ]:
accept_all = accept_temp * accept_ohc * accept_co2 #* accept_faer
#accept_all = accept_temp * accept_co2
valid_all = np.arange(samples, dtype=int)[accept_all]
np.sum(accept_all)

## Everything!

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.min(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), axis=1), 
    np.max(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.percentile(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), 5, axis=1), 
    np.percentile(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850.5, 2031), 
    np.percentile(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), 16, axis=1), 
    np.percentile(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1850.5, 2031), 
    np.median(temp_in[:,accept_all]-temp_in[:51,accept_all].mean(axis=0), axis=1), 
    color='#000000',
)
ax.plot(np.arange(1850.5, 2021), gmst, color='b')
ax.set_xlim(1850,2030)
ax.set_ylim(-1, 3)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Temperature anomaly - constrained')

In [ ]:
pl.hist(temp_in[145:165,accept_all].mean(axis=0)-temp_in[:51,accept_all].mean(axis=0))
np.percentile(temp_in[145:165,accept_all].mean(axis=0)-temp_in[:51,accept_all].mean(axis=0), (5, 50, 95))

In [ ]:
# target is 329 396 463 for 16, 50, 84
pl.hist(ohc_to_eshc_factor*(ohc_in[-1, accept_all]-ohc_in[0, accept_all]));
np.percentile(ohc_to_eshc_factor*(ohc_in[-1, accept_all]-ohc_in[0, accept_all]), (5, 16, 50, 84, 95))

In [ ]:
valid_all

In [ ]:
np.median(temp_in[180,accept_all]-temp_in[:51,accept_all].mean(axis=0))

In [ ]:
fig, ax = pl.subplots()
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.min(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), axis=1), 
    np.max(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), axis=1),
    color='#000000',
    alpha=0.03,
)
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), 5, axis=1), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1971.5, 2019), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), 16, axis=1), 
    np.percentile(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1971.5, 2019), 
    np.median(ohc_to_eshc_factor*(ohc_in[:, accept_all]-ohc_in[0, accept_all]), axis=1), 
    color='#000000',
)
ax.plot(np.arange(1971.5, 2019), ohc*1e21, color='b')
ax.set_xlim(1971,2019)
#ax.set_ylim(-1, 3)
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.title('Ocean heat content - constrained')

In [ ]:
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_scale = pd.read_csv('../data/parameter_sets/forcing_scaling.csv')

In [ ]:
# use F2xCO2 and scale factor to determine ECS: this is what each climate simulation actually "sees"
erf_2co2 = 3.934168323890023

In [ ]:
ecs = np.ones(samples) * np.nan
tcr = np.ones(samples) * np.nan
for i in tqdm(valid_all):
    ebm = EnergyBalanceModel(
        ocean_heat_capacity = np.array([df_cr.loc[i,'c1'], df_cr.loc[i,'c2'], df_cr.loc[i, 'c3']]),
        ocean_heat_transfer = np.array([df_cr.loc[i,'kappa1'], df_cr.loc[i,'kappa2'], df_cr.loc[i,'kappa3']]),
        deep_ocean_efficacy = df_cr.loc[i,'epsilon'],
        forcing_4co2 = 2*erf_2co2*df_scale.loc[i,'CO2'],
    )
    ebm.emergent_parameters()
    ecs[i] = ebm.ecs
    tcr[i] = ebm.tcr

In [ ]:
pl.hist(ecs[~np.isnan(ecs)]);
np.percentile(ecs[~np.isnan(ecs)], (5, 16, 50, 84, 95))

In [ ]:
pl.hist(tcr[~np.isnan(tcr)]);
np.percentile(tcr[~np.isnan(tcr)], (5, 16, 50, 84, 95))

In [ ]:
# target -2.0 -1.3 -0.6
pl.hist(faer_in[5:15, accept_all].mean(axis=0))
np.percentile(faer_in[5:15, accept_all].mean(axis=0), (5, 16, 50, 84, 95))

In [ ]:
pl.hist(faer_in[19, accept_all])
np.percentile(faer_in[19, accept_all], (5, 16, 50, 84, 95))

In [ ]:
np.sum(faer_in[5:15, accept_all].mean(axis=0)>-2.2)

In [ ]:
pl.plot(fari_in[:, accept_all].mean(axis=1))

In [ ]:
pl.plot(faci_in[:, accept_all].mean(axis=1))

In [ ]:
np.savetxt('../data/ar6_ensemble_batches/accepted.csv', valid_all)

In [ ]:
pl.hist(df_cr[:samples].loc[accept_all,'kappa1']);
np.percentile(df_cr[:samples].loc[accept_all,'kappa1'], (5, 16, 50, 84, 95))